In [119]:
import json
import os
import ast
import numpy as np 
import networkx as nx

In [ ]:
#
RAW_DIR = "../raw/"
LIST_RAW_CSV = "../list_raw/all.csv"
LIST_RAW_JSON = "../transformed/all_json.json"
TRANSFORMED_DIR = "../transformed/"
OUTPUT = "output.txt"
GBTOUTPUT = "gbt.txt"
TITLEOUTPUT = "title.txt"
full_keys = ["author","title","journal","year","DOI","month","citations(google scholar)","abstract","keywords","reference_count","ccfClass","important","references"]
full_keys_default = {"author":[],"title":"","journal":"","year":0,"DOI":"","month":0,"citations(google scholar)":-1,"abstract":"","keywords":[],"reference_count":0,"ccfClass":"","important":None,"references":[]}
# 常用函数定义
# 保存至transform
def save_tran_json(addr,jsonfile):
    with open(TRANSFORMED_DIR + addr, 'w') as o:
        o.write(json.dumps(jsonfile,ensure_ascii=False))
        
def save_tran_plain(addr,txtfile):
    with open(TRANSFORMED_DIR + addr, 'w') as o:
        o.write(txtfile)
        
def save_tran_lines(addr,txtfile):
    with open(TRANSFORMED_DIR + addr, 'a+') as o:
        for line in txtfile:
            o.write(line+'\n')    
# 从transformd中读取
def read_tran_json(addr):
    with open(TRANSFORMED_DIR + addr,encoding = 'utf-8') as f:
        data = json.load(f)
    return data
    
#  查询ccf等级
def search_ccf(s):
    ccf_all_addr = "/Users/Halloween/Desktop/Study/复杂网络/lrcns/raw-data/ccf/ccf_all.csv"

    ccf_all = []
    ccf_search_res = {}
    with open(ccf_all_addr,encoding = 'utf-8') as f:
        reader = csv.reader(f)
        for line in reader:
            ccf_all.append(line)
    level = ""
    field = ""
    cORj = ""
    col = -1
#     print(ccf_all)
    for line in reversed(ccf_all):
        if col != 1 and col != 2 and col != -1:
            col = -1
            continue
        if col == -1:
            for word in line[1:3]:
                if word == "":
                    continue
                if(string_similar(word,s)>0.9):
                    col = line.index(word)
                    ccf_search_res['ccf_search_res'] = word
#                     print('"'+word.strip('"')+'"'+'"'+s.strip('"')+'"')
#                     print(col)
#                     print(line)
                    break
        else:
            if len(line[0])>1 and line[0][1]=='、' and level == "":
                level = line[0][2]
#                 print(level)
            if len(line[0])>10 and line[0][:2]=="中国" and field == "":
                field = line[0][line[0].find("（")+1:line[0].find("）")]
                cORj = line[0][line[0].find("术")+1:line[0].find("（")]
#                 print(field)
    ccf_search_res.update({"level":level,"field":field,"type":cORj})
    return {"level":level,"field":field,"type":cORj}
# 数据审查函数
def inspect(to_inspect):
    papers_arr = to_inspect
    all_error = 0
    for paper in papers_arr:
        inspect = []
        for key in full_keys:
            if(key not in paper.keys() or type(paper[key]) != type(full_keys_default[key]) or paper[key] == full_keys_default[key]):
                inspect.append(key + " lost")
                continue
            if(key == "author" or key == "keywords" or key == "references"):
                if(type(paper[key]) != list or len(paper[key])==0):
                    inspect.append(key + " lost")
                    continue
            if(key == "year"):
                if(paper[key]<1900):
                    inspect.append(key + " error")
            if(key == "month"):
                if(paper[key]<0 or paper[key] >12):
                    inspect.append(key + " error")
            if(key == "citations(google scholar)" or key == "reference_count"):
                if(paper[key]<0):
                    inspect.append(key + " error")
            if(key == 'ccfClass'):
                if(paper[key] not in ['A','B','C']):
                    inspect.append(key + " error")
            if(type(paper[key])==int):
                if(paper[key]<0):
                    inspect.append(key + " less than 0")
                continue
            if(type(paper[key])==float):
                inspect.append(key + " not int")
                continue
        paper['inspect'] = list(inspect)
        print(paper['title']+str(paper['inspect']))
        all_error += len(paper['inspect'])
    print(">>>>SUMMARY ERRORs:"+str(all_error)+"<<<<<")
    return papers_arr

In [87]:
# parse sth like DOI [DOI 10.****] or DOI DOI 10.***
def parseDOI(doi):
    return [d.strip(' ').strip('[').strip(']').strip('DOI ').lower() for d in doi.strip(' ').strip('[').strip(']').strip('DOI ').split(',')]

def parseWoS(PATH,DEST):
    # "key":[type,de]
    FIELD_TYPE = {"PT":[str],"AU":[list,'\n'],"AF":[list,'\n'],"BA":[list,'\n'],"BF":[list,'\n'],"CA":[list,'\n'],"GP":[list,'\n'],"BE":[list,'\n'],"TI":[str],"SO":[str],"SE":[str],"BS":[str],"LA":[str],"DT":[str],"CT":[str],"CY":[str],"CL":[str],"SP":[list,'\n'],"HO":[str],"DE":[list,';'],"ID":[list,';'],"AB":[str],"C1":[str],"RP":[str],"EM":[list,';'],"RI":[list,';'],"OI":[list,';'],"FU":[list,';'],"FX":[str],"CR":[list,'\n'],"NR":[int],"TC":[int],"Z9":[int],"U1":[int],"U2":[int],"PU":[list,','],"PI":[str],"PA":[str],"SN":[str],"EI":[str],"BN":[str],"J9":[str],"JI":[str],"PD":[str],"PY":[int],"VL":[str],"IS":[str],"SI":[str],"PN":[str],"SU":[str],"MA":[str],"BP":[str],"EP":[str],"AR":[str],"DI":[str],"D2":[str],"EA":[str],"EY":[str],"PG":[str],"P2":[str],"WC":[list,';'],"SC":[list,';'],"GA":[str],"PM":[str],"UT":[str],"OA":[str],"HP":[str],"HC":[str],"DA":[str]}
    # 清空文件
    open(DEST,'w').close()
    wos_arr = []
    addr_list = []
    for addr in os.walk(PATH):
        for fname in addr[2]:
            if(fname[0]!='.' and fname.split('.')[-1]=="txt"):
                addr_list.append(str(addr[0]+"/"+fname))

                
    for addr in addr_list:
        print(addr)
#         单文件最多500条，可以读入内存
        with open(addr,encoding = 'utf-8-sig', errors = 'ignore') as f:
            fstrs = f.readlines()  
        tg_key = {}        
        tag = ""
        tag_type = [""]
        for line in fstrs:
            if(line[:2] == "EF"):
#                 结束本文件
                break
            elif(line[:2] == "ER"):
#                 if('WC' in tg_key.keys()):
#                     print(tg_key['WC'])
#             存放记录
                wos_arr.append(dict(tg_key))
                tg_key = {}        
                tag = ""
                tag_type = [""]
            elif(line[:2] != "  " and line[:2] in FIELD_TYPE.keys()):
#                 找到一个新filed
                if(tag_type[0]==list):
#                 上一个
                    tg_key[tag] = [x.replace('\n','').strip(' ') for x in list(tg_key[tag].strip('\n').split(tag_type[1]))]
#                     if(tag == 'WC'):
#                         print(tg_key[tag])
# #                         print(tg_key['WC'])
                if 'CR' == tag:
                    tg_key_CR = []
                    for cr in tg_key['CR']:
                        DOI_loc = cr.find("DOI")
                        if (DOI_loc != -1):
                            ref = cr[:DOI_loc].strip(', ')
                            DOI = parseDOI(cr[DOI_loc:])
                            tg_key_CR.append({"ref":ref,"DOI":DOI})
                        else:
                            tg_key_CR.append({"ref": cr,"DOI":[""]})
                    tg_key['CR'] = tg_key_CR
    
                tag = line[:2]
                tag_type = FIELD_TYPE[tag]
                if(tag_type[0]==str):
                    tg_key[tag] = ""
                    if(tag=="DI"):
                        tg_key[tag] += line[3:].lower().strip('\n')
                    else:
                        tg_key[tag] += line[3:].strip('\n')
                elif(tag_type[0]==int):
                    tg_key[tag] = int(line[3:].strip('\n'))
                elif(tag_type[0]==list):
                    tg_key[tag] = ""
                    tg_key[tag] += line[3:]
                    if (tag_type[1] != '\n'):
                        tg_key[tag] = tg_key[tag].strip('\n')
            elif(line[:2] == "  "):
#                 追加

                if(tag_type[0]==str):
                    tg_key[tag] += ' '
                    tg_key[tag] += line[3:].strip('\n')
                elif(tag_type[0]==list):
                    if (tag_type[1] != '\n'):
                        tg_key[tag] += ' '
                        tg_key[tag] = tg_key[tag].strip('\n')
                    tg_key[tag] += line[3:]

    print(len(wos_arr))
    with open(DEST,'a+') as o:
        o.write(json.dumps(wos_arr, ensure_ascii=False))
        



In [88]:
parseWoS("/Users/Halloween/Downloads/saved500","/Users/Halloween/Desktop/Study/复杂网络/lrcns/raw-data/transformed/wos2.txt")

/Users/Halloween/Downloads/saved500/savedrecs (1).txt
/Users/Halloween/Downloads/saved500/savedrecs (11).txt
/Users/Halloween/Downloads/saved500/savedrecs (10).txt
/Users/Halloween/Downloads/saved500/savedrecs (7).txt
/Users/Halloween/Downloads/saved500/savedrecs (6).txt
/Users/Halloween/Downloads/saved500/savedrecs .txt
/Users/Halloween/Downloads/saved500/savedrecs (9).txt
/Users/Halloween/Downloads/saved500/savedrecs (5).txt
/Users/Halloween/Downloads/saved500/savedrecs (4).txt
/Users/Halloween/Downloads/saved500/savedrecs (8).txt
/Users/Halloween/Downloads/saved500/savedrecs (3).txt
/Users/Halloween/Downloads/saved500/savedrecs (2).txt
5600


In [89]:
wos_arr = read_tran_json('wos2.txt')
print(len(wos_arr))

5600


In [90]:
print(wos_arr[0]['CR'])

[{'ref': 'Adamic LA, 2003, SOC NETWORKS, V25, P211', 'DOI': ['10.1016/s0378-8733(03)00009-1']}, {'ref': 'Akoglu L, 2015, DATA MIN KNOWL DISC, V29, P626', 'DOI': ['10.1007/s10618-014-0365-y']}, {'ref': 'Bayati M, 2009, IEEE DATA MINING, P705', 'DOI': ['10.1109/icdm.2009.135']}, {'ref': 'Belkin M, 2002, ADV NEUR IN, V14, P585', 'DOI': ['']}, {'ref': 'Belkin M, 2003, NEURAL COMPUT, V15, P1373', 'DOI': ['10.1162/089976603321780317']}, {'ref': 'Benson AR, 2016, SCIENCE, V353, P163', 'DOI': ['10.1126/science.aad9029']}, {'ref': 'Berline N., 2003, HEAT KERNELS DIRAC O', 'DOI': ['']}, {'ref': 'Bollacker K, 2008, P 2008 ACM SIGMOD IN, P1247', 'DOI': ['10.1145/1376616.1376746', '10.1145/1376616.1376746']}, {'ref': "Bourigault S, 2014, WSDM'14: PROCEEDINGS OF THE 7TH ACM INTERNATIONAL CONFERENCE ON WEB SEARCH AND DATA MINING, P393", 'DOI': ['10.1145/2556195.2556216']}, {'ref': 'Breitkreutz BJ, 2008, NUCLEIC ACIDS RES, V36, pD637', 'DOI': ['10.1093/nar/gkm1001']}, {'ref': 'Burt RS, 2004, AM J SOCI

In [78]:
# 编号
# 所有paper 含有 title 以此编号，再将编号对应doi
# 识别时，通过TI 识别paper
ti_set = set([x['TI'] for x in wos_arr if 'TI' in x.keys()])
ti_to_paper = {x['TI']:x for x in wos_arr if 'TI' in x.keys()}
idx_to_ti = {i:t for i,t in enumerate(title_set)}
ti_to_idx = {t:i for i,t in enumerate(title_set)}
# DOI 根据title编号
doi_set = set([x['DI'] for x in wos_arr if 'DI' in x.keys()])

doi_to_paper = {x['DI']:x for x in wos_arr if 'DI' in x.keys()}
idx_to_doi = {ti_to_idx[paper['TI']]:paper['DI'] for paper in wos_arr if 'DI' in paper.keys() }
doi_to_idx = {paper['DI']:ti_to_idx[paper['TI']] for paper in wos_arr if 'DI' in paper.keys() }

print(len(idx_to_doi))

3735


In [50]:
# author 编号
author_set = set([author for paper in wos_arr if 'AF' in paper.keys() for author in paper['AF'] ])
idx_to_af = {i:a for i,a in enumerate(author_set)}
af_to_idx = {a:i for i,a in enumerate(author_set)}
print(len(idx_to_af))

15309


In [105]:
# 只在引文ref rdoi (all doi) 
rdoi_set = {ref['DOI'][0] for paper in wos_arr if 'CR' in paper.keys() for ref in paper['CR'] if ref['DOI'][0] != "" and ref['DOI'][0] not in doi_set}
idx_to_rdoi = {i+len(ti_set):t for i,t in enumerate(rdoi_set)}
rdoi_to_idx = {t:i+len(ti_set) for i,t in enumerate(rdoi_set)}
# 所有doi
adoi_set = rdoi_set.union(doi_set)
idx_to_adoi = dict()
idx_to_adoi.update(idx_to_rdoi)
idx_to_adoi.update(idx_to_doi)
adoi_to_idx = dict()
adoi_to_idx.update(doi_to_idx)
adoi_to_idx.update(rdoi_to_idx)
print(len(doi_set))
print(len(adoi_set))
print(len(rdoi_set))
print(len({ref['DOI'][0] for paper in wos_arr if 'CR' in paper.keys() for ref in paper['CR'] if ref['DOI'][0] != ""}))

3743
65000
61257
62237


In [96]:
# 构造矩阵
WA = np.zeros((len(idx_to_ti),len(idx_to_af)))
for paper in wos_arr:
    if 'AF' in paper.keys():
        for author in paper['AF']:
            WA[ti_to_idx[paper['TI']],af_to_idx[author]] = 1
print(WA.shape)
AW = WA.T            

(5541, 15309)


In [110]:
# (文章数，文章数+非文章DOI的引用DOI)
WR = np.zeros((len(idx_to_ti),len(idx_to_ti) + len(idx_to_rdoi)))
for paper in wos_arr:
    if 'CR' in paper.keys():
        for ref in paper['CR']:
            if ref['DOI'][0] != "" :
                if ref['DOI'][0] in rdoi_to_idx.keys():
                    WR[ti_to_idx[paper['TI']],rdoi_to_idx[ref['DOI'][0]]] = 1
                elif ref['DOI'][0] in doi_to_idx.keys():
                    WR[ti_to_idx[paper['TI']],doi_to_idx[ref['DOI'][0]]] = 1
print(WR.shape)

(5541, 66798)


In [111]:
WRW = np.dot(WR,WR.T)
print(WRW.shape)

(5541, 5541)


In [133]:
G = nx.from_numpy_matrix(WRW)

In [132]:
np.savetxt(TRANSFORMED_DIR+'matrix.csv',WRW,fmt="%d",delimiter=",")

[[0, 1, 1], [1, 0, 0]]


In [70]:
WAW = np.dot(WA,AW)
print(WAW.shape)

(5541, 5541)


In [71]:
print(WW)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 2. 0. ... 0. 0. 0.]
 [0. 0. 3. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 5. 0. 0.]
 [0. 0. 0. ... 0. 6. 0.]
 [0. 0. 0. ... 0. 0. 4.]]


In [ ]:
author_set = set([x['DI'] for x in wos_arr if 'DI' in x.keys()])

In [ ]:
print(idx_to_doi)
print(doi_to_idx)

In [ ]:
print(doi_to_paper[idx_to_doi[1]])

In [ ]:
wc_set = set()
wc_dict = {}
for paper in wos_arr:
    if('WC' in paper.keys()):
#         wc_set.update(paper['WC'])
        for wc in paper['WC']:
            if(wc not in wc_dict):
                wc_dict[wc] = 1
            else:
                wc_dict[wc] += 1
print(wc_dict)

In [ ]:
for paper in wos_arr:
    if("DI" in paper.keys()):
        print(paper["WC"])

In [ ]:
doi_cnt = 0
for paper in wos_arr:
    if("DI" in paper.keys()):
        print(paper['DI'],end="")
        if("CR" in paper.keys()):
            for ref in paper['CR']:
                if(ref["DOI"][0]!=""):
                    print(";"+ref["DOI"][0],end = "")
        print("")
        
print(doi_cnt)

In [ ]:
cnt = 0
for paper in wos_arr:
    try:
        for sentence in (paper['AB'].split('.')):
            if "propose" in sentence:
                print(sentence)
    except:
        cnt += 1
        
print(cnt)